In [9]:
import pandas_datareader as web
import datetime
import numpy as np
import pandas as pd
from time import sleep


In [4]:
START_YR = "2009"
YRS_MIN = 10
RAW_DIR = "data/raw"
YRS_DIR = "data/yearly"
FORCE_UPDATE = False
REPASS = False
STOCK_LIST_DIR = "stock_list"
START_LIST_NUM = 0
END_LIST_NUM = 0 #(500/4)
BOT_MAX = 4
BOT_NUM = 1

def set_col(df, c_name, c_type="str", tmp_i="False", v=np.nan):
    if not (c_name in df.columns):
        if c_type == "str":
            df[c_name] = ""
            df[c_name] = df[c_name].astype(str)
        else:
            df[c_name] = np.nan
    if not tmp_i == "False":
        if c_type == "str":
            try:
                df.at[tmp_i, c_name] = ""
            except:
                df[c_name] = ""
                
        df.at[tmp_i, c_name] = v
        

try:
    tickers = pd.read_csv(r'{}.csv'.format(STOCK_LIST_DIR), engine='python')
    tickers = tickers.loc[:, ~tickers.columns.str.contains('^Unnamed')]
except:
    pass
    tickers = pd.DataFrame()


set_col(tickers, 'symbol')
set_col(tickers, 'company')
set_col(tickers, 'price', "int")
set_col(tickers, 'ignore')
set_col(tickers, 'start_yr')
set_col(tickers, 'ignore_error')
# set_col(tickers, 'adj_up_start_schedule')
# set_col(tickers, 'adj_up_end_schedule')

if tickers.empty:
    tickers.to_csv(r'{}.csv'.format(STOCK_LIST_DIR))


for i, row in tickers.iterrows():
    if START_LIST_NUM>0 and i<START_LIST_NUM:
        continue
    if END_LIST_NUM>0 and END_LIST_NUM>START_LIST_NUM and i>END_LIST_NUM:
        break
    SYMBOL = row["symbol"]
#     SYMBOL = "PG"
    
    try:        
        current_stock = pd.read_csv(r'{}/{}_{}.csv'.format(RAW_DIR, SYMBOL, "history"), index_col=['Date'] , parse_dates=['Date'])
        tickers.at[i, "ignore"] = False
    except:
        current_stock = ""
        
    print(i+1, SYMBOL)
    if (pd.notnull(row['ignore']) and row['ignore'] and not REPASS):
        print("--Ignore: {}".format(row['ignore'] and not REPASS))
        continue
    if FORCE_UPDATE or not str(current_stock):
        print("--get data")
        try:
#             sleep(15)
            current_stock = web.DataReader(SYMBOL, data_source="yahoo", start=START_YR) 
            set_col(tickers, "ignore","str", i, False)
        except Exception as e:
            set_col(tickers, "ignore","str", i, True)
            set_col(tickers, "ignore_error","str", i, e)
            print("---Error: {}".format(e))
            tickers.to_csv(r'{}.csv'.format(STOCK_LIST_DIR))
            continue
    
        

    tickers.at[i, "price"] = current_stock.tail(1)["Close"][0]
    tickers.at[i, "ignore"] = skip = current_stock.tail(1).index[0].year-current_stock.head(1).index[0].year < YRS_MIN
    tickers.at[i, "start_yr"] = current_stock.head(1).index[0].year
    print("--Skip: {}".format(skip))
    if skip:
        current_stock.to_csv(r'{}/{}_{}.csv'.format("data/under_min", current_stock.head(1).index[0].year, SYMBOL))
        tickers.to_csv(r'{}.csv'.format(STOCK_LIST_DIR))
        continue
    else:
        current_stock.to_csv(r'{}/{}_{}.csv'.format(RAW_DIR, SYMBOL, "history"))


    yearly_adj_df = pd.DataFrame()
    yearly_df = pd.DataFrame()
    for j, row2 in current_stock.iterrows():
        yearly_adj_df.at["{:02d}-{:02d}".format(j.month, j.day), j.year] = row2["Adj Close"]
        yearly_df.at["{:02d}-{:02d}".format(j.month, j.day), j.year] = row2["Close"]
  
    yearly_adj_df.sort_index(inplace=True)
    yearly_df.sort_index(inplace=True)

    r_adj_df_list = {}
    r_c_df_list = {}
    r_pc_df_list = {}
    d_range = list(range(76))[0::15][1:][::-1]
    for r in d_range:
        r_adj_df_list[r] = pd.DataFrame()
        r_c_df_list[r] = pd.DataFrame()
        r_pc_df_list[r] = pd.DataFrame()
        for j, row2 in yearly_df.iterrows():
            adj_d_ups = adj_d_downs = c_d_ups = c_d_downs = 0
            for yr in yearly_df.columns:
                d = pd.to_datetime('{}-{}'.format(yr, j), format="%Y-%m-%d", dayfirst=True, errors='coerce')
                r_adj_df_list[r].at[j, yr] = r_c_df_list[r].at[j, yr] = r_pc_df_list[r].at[j, yr] = 0
                if not pd.isnull(d):
                    f = d + datetime.timedelta(days=r)
                    tmp_r_df = current_stock.loc['{}-{:02d}-{:02d}'.format(d.year, d.month, d.day):'{}-{:02d}-{:02d}'.format(f.year, f.month, f.day)]

                    if not tmp_r_df.empty:
                        adj_first_range_num = tmp_r_df.head(3)["Adj Close"].dropna().head(1)[0]
                        c_first_range_num = tmp_r_df.head(3)["Close"].dropna().head(1)[0]              
                        adj_last_range_num = tmp_r_df.tail(3)["Adj Close"].dropna().head(1)[0]
                        c_last_range_num = tmp_r_df.tail(3)["Close"].dropna().head(1)[0]

                        r_adj_df_list[r].at[j, yr] = adj_last_range_num - adj_first_range_num
                        r_c_df_list[r].at[j, yr] = c_last_range_num - c_first_range_num
                        r_pc_df_list[r].at[j, yr] = ((c_last_range_num - c_first_range_num)/c_last_range_num)*100
                        if (adj_last_range_num - adj_first_range_num)>0:
                            adj_d_ups=adj_d_ups+1
                        else:
                            adj_d_downs=adj_d_downs+1

                        if (c_last_range_num - c_first_range_num)>0:
                            c_d_ups=c_d_ups+1
                        else:
                            c_d_downs=c_d_downs+1


            set_col(r_adj_df_list[r], "ups", "int", j, adj_d_ups)
            set_col(r_c_df_list[r], "ups", "int", j, c_d_ups)
            set_col(r_pc_df_list[r], "ups", "int", j, c_d_ups)
            set_col(tickers, "adj_ups_r_{}".format(r), "int", i, adj_d_ups)
            set_col(tickers, "close_ups_r_{}".format(r), "int", i, c_d_ups)
            set_col(r_adj_df_list[r], "downs", "int", j, adj_d_downs)
            set_col(r_c_df_list[r], "downs", "int", j, c_d_downs)
            set_col(r_pc_df_list[r], "downs", "int", j, c_d_downs)
            set_col(tickers, "adj_downs_r_{}".format(r), "int", i, adj_d_downs)
            set_col(tickers, "close_downs_r_{}".format(r), "int", i, c_d_downs)
            set_col(r_adj_df_list[r], "end_day_sum","str", j, '{:02d}-{:02d}'.format(f.month, f.day))
            set_col(r_c_df_list[r], "end_day_sum","str", j, '{:02d}-{:02d}'.format(f.month, f.day))
        
        r_adj_df_list[r]['avg'] = r_adj_df_list[r].loc[:, ~r_adj_df_list[r].columns.str.contains('^ups|^downs|^avg|^end_day_sum', na=False)].mean(axis=1)
        r_c_df_list[r]['avg'] = r_c_df_list[r].loc[:, ~r_c_df_list[r].columns.str.contains('^ups|^downs|^avg|^end_day_sum', na=False)].mean(axis=1)
        r_pc_df_list[r]['avg'] = r_pc_df_list[r].loc[:, ~r_pc_df_list[r].columns.str.contains('^ups|^downs|^avg|^end_day_sum', na=False)].mean(axis=1)

        range_dir = "data/r_{}".format(r)
        if not os.path.exists(range_dir)
            os.mkdir(range_dir)
        r_adj_df_list[r].sort_index(inplace=True)
        r_adj_df_list[r].to_csv(r'{}/{}_r{}_adj_data.csv'.format(range_dir, SYMBOL, r))
        r_c_df_list[r].sort_index(inplace=True)
        r_c_df_list[r].to_csv(r'{}/{}_r{}_c_data.csv'.format(range_dir, SYMBOL, r))
        r_pc_df_list[r].sort_index(inplace=True)
        r_pc_df_list[r].to_csv(r'{}/{}_r{}_pc_data.csv'.format(range_dir, SYMBOL, r))

        set_col(tickers, "adj_up_start_schedule_r_{}".format(r),"str", i, r_adj_df_list[r].sort_values(['ups', 'avg'], ascending=[False, False]).head(1).index[0])
        set_col(tickers, "adj_up_end_schedule_r_{}".format(r),"str", i, r_adj_df_list[r].sort_values(['ups', 'avg'], ascending=[False, False]).head(1)["end_day_sum"][0])
        set_col(tickers, "close_up_start_schedule_r_{}".format(r),"str", i, r_c_df_list[r].sort_values(['ups', 'avg'], ascending=[False, False]).head(1).index[0])
        set_col(tickers, "close_up_end_schedule_r_{}".format(r),"str", i, r_c_df_list[r].sort_values(['ups', 'avg'], ascending=[False, False]).head(1)["end_day_sum"][0])
        set_col(tickers, "close_up_percent_avg_r_{}".format(r),"int", i, r_pc_df_list[r].sort_values(['ups', 'avg'], ascending=[False, False]).head(1)["avg"][0])
        
    yearly_adj_df.to_csv(r'{}/{}_yrs_adj_data.csv'.format(YRS_DIR, SYMBOL))
    yearly_df.to_csv(r'{}/{}_yrs_close_data.csv'.format(YRS_DIR, SYMBOL))
    tickers.to_csv(r'{}.csv'.format(STOCK_LIST_DIR))
print("Done")

1 WYNYF
--get data
---Error: No data fetched for symbol WYNYF using YahooDailyReader
2 WYNN
--get data
--Skip: False
3 WYNMY
--get data
--Skip: False
4 WYNMF
--get data
--Skip: False
5 WH
--get data
--Skip: True
6 WYND
--get data
--Skip: False
7 WW
--get data
--Skip: False
8 WVFC
--get data
--Skip: False
9 WUYIF
--get data
---Error: No data fetched for symbol WUYIF using YahooDailyReader
10 WX
--get data
---Error: 'Date'
11 WXIBF
--get data
--Skip: True
12 WXXWY
--get data
--Skip: True
13 WUXIF
--get data
--Skip: True
14 WUMSY
--get data
---Error: No data fetched for symbol WUMSY using YahooDailyReader
15 WUMSF
--get data
--Skip: True
16 WBWB
--get data
--Skip: True
17 WTBFB
--get data
--Skip: False
18 WTBFA
--get data
--Skip: True
19 WSPOF
--get data
--Skip: True
20 WSCI
--get data
--Skip: True
21 WSFS
--get data
--Skip: False
22 WRIT
--get data
--Skip: False
23 WMGI
--get data
--Skip: True
24 IWSH
--get data
--Skip: True
25 WRTC
--get data
--Skip: True
26 WRB
--get data
--Skip: False

In [14]:
#     for three months 70-75
# def d_range(current_stock, yearly_df, tickers, i):
# return {'range':r,}
    
#     three_m_adj_df = pd.DataFrame()
#     three_m_c_df = pd.DataFrame()
#     three_m_pc_df = pd.DataFrame()